In [ ]:
## 1. 라이브러리 불러오기

import pickle
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from sklearn.preprocessing import MinMaxScaler
import time

# 시각화 설정
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set(font_scale=1.2)

print("기본 라이브러리 로드 완료")

In [ ]:
## 2. 전처리된 데이터 로드

# 데이터 경로 설정
PROCESSED_DIR = "./data/processed"
TRUST_DIR = "./data/trust"

# 디렉토리 생성
os.makedirs(TRUST_DIR, exist_ok=True)

# 필요한 데이터 파일 로드
data_files = {
    'interaction_df': 'interaction_df.pkl',
    'train_df': 'train_df.pkl',
    'val_df': 'val_df.pkl',
    'test_df': 'test_df.pkl',
    'user_processed': 'user_processed.pkl',
    'business_processed': 'business_processed.pkl',
    'tip_processed': 'tip_processed.pkl'
}

# 데이터 로드
data = {}
for name, file in data_files.items():
    with open(os.path.join(PROCESSED_DIR, file), 'rb') as f:
        data[name] = pickle.load(f)
    print(f"{name} 로드 완료")

# 데이터 변수에 할당
interaction_df = data['interaction_df']
train_df = data['train_df']
val_df = data['val_df']
test_df = data['test_df']
user_processed = data['user_processed']
business_processed = data['business_processed']
tip_processed = data['tip_processed']

print(f"훈련 세트 크기: {len(train_df):,}")
print(f"사용자 수: {len(user_processed):,}")
print(f"비즈니스 수: {len(business_processed):,}")

In [ ]:
## 3. 기본 사용자 정보 탐색

# 사용자 활동 정보 기본 통계
print("\n=== 사용자 활동 통계 ===")
print("리뷰 수 통계:")
print(user_processed['review_count'].describe())

print("\n팬 수 통계:")
print(user_processed['fans'].describe())

print("\n평균 별점 통계:")
print(user_processed['average_stars'].describe())

# 컴플리먼트 관련 컬럼
compliment_cols = [col for col in user_processed.columns if 'compliment_' in col]
print(f"\n컴플리먼트 컬럼 ({len(compliment_cols)}개): {compliment_cols}")

# 총 컴플리먼트 통계
print("\n총 컴플리먼트 통계:")
print(user_processed['total_compliments'].describe())

# 활동 강도 통계
print("\n활동 강도 (리뷰 수 / 활동 기간) 통계:")
print(user_processed['activity_intensity'].describe())

# 히스토그램 시각화
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 리뷰 수 분포
sns.histplot(user_processed['review_count'], bins=50, kde=True, ax=axes[0, 0])
axes[0, 0].set_title('사용자별 리뷰 수 분포')
axes[0, 0].set_xlabel('리뷰 수')
axes[0, 0].set_ylabel('사용자 수')
axes[0, 0].set_xlim(0, 500)  # 가독성을 위해 x축 제한

# 팬 수 분포
sns.histplot(user_processed['fans'], bins=50, kde=True, ax=axes[0, 1])
axes[0, 1].set_title('사용자별 팬 수 분포')
axes[0, 1].set_xlabel('팬 수')
axes[0, 1].set_ylabel('사용자 수')
axes[0, 1].set_xlim(0, 50)  # 가독성을 위해 x축 제한

# 총 컴플리먼트 분포
sns.histplot(user_processed['total_compliments'], bins=50, kde=True, ax=axes[1, 0])
axes[1, 0].set_title('사용자별 총 컴플리먼트 분포')
axes[1, 0].set_xlabel('총 컴플리먼트 수')
axes[1, 0].set_ylabel('사용자 수')
axes[1, 0].set_xlim(0, 500)  # 가독성을 위해 x축 제한

# 활동 강도 분포
sns.histplot(user_processed['activity_intensity'], bins=50, kde=True, ax=axes[1, 1])
axes[1, 1].set_title('사용자별 활동 강도 분포')
axes[1, 1].set_xlabel('활동 강도 (리뷰 수 / 활동 기간)')
axes[1, 1].set_ylabel('사용자 수')
axes[1, 1].set_xlim(0, 0.5)  # 가독성을 위해 x축 제한

plt.tight_layout()
plt.show()

In [ ]:
## 4. 리뷰 품질 지표 분석

# 리뷰 유용성 지표 (useful, funny, cool) 분석
review_reaction_cols = ['useful', 'funny', 'cool']

if all(col in interaction_df.columns for col in review_reaction_cols):
    print("\n=== 리뷰 반응 통계 ===")
    for col in review_reaction_cols:
        print(f"\n{col.capitalize()} 통계:")
        print(interaction_df[col].describe())
    
    # 리뷰 반응 합계 계산
    interaction_df['total_reactions'] = interaction_df[review_reaction_cols].sum(axis=1)
    
    # 리뷰 반응 분포 시각화
    plt.figure(figsize=(12, 6))
    sns.histplot(interaction_df['total_reactions'], bins=50, kde=True)
    plt.title('리뷰별 총 반응 수 분포')
    plt.xlabel('총 반응 수 (useful + funny + cool)')
    plt.ylabel('빈도')
    plt.xlim(0, 50)  # 가독성을 위해 x축 제한
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("리뷰 반응 컬럼(useful, funny, cool)이 데이터에 없습니다.")

# 리뷰 텍스트 길이와 유용성 관계 분석
if 'text_length' in interaction_df.columns and 'useful' in interaction_df.columns:
    # 리뷰 길이와 유용성 점수의 산점도 (샘플링)
    plt.figure(figsize=(12, 6))
    sample_size = min(5000, len(interaction_df))
    sample_indices = np.random.choice(len(interaction_df), sample_size, replace=False)
    
    sns.scatterplot(
        x='text_length', 
        y='useful', 
        data=interaction_df.iloc[sample_indices],
        alpha=0.5
    )
    plt.title('리뷰 길이와 유용성 점수의 관계')
    plt.xlabel('리뷰 텍스트 길이 (단어 수)')
    plt.ylabel('유용성 점수')
    plt.xlim(0, 500)  # 가독성을 위해 x축 제한
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # 리뷰 길이 구간별 평균 유용성 점수
    interaction_df['length_bin'] = pd.cut(
        interaction_df['text_length'], 
        bins=[0, 50, 100, 200, 300, 500, 1000, np.inf], 
        labels=['0-50', '51-100', '101-200', '201-300', '301-500', '501-1000', '1000+']
    )
    
    plt.figure(figsize=(12, 6))
    sns.barplot(x='length_bin', y='useful', data=interaction_df)
    plt.title('리뷰 길이 구간별 평균 유용성 점수')
    plt.xlabel('리뷰 텍스트 길이 (단어 수)')
    plt.ylabel('평균 유용성 점수')
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("리뷰 텍스트 길이 또는 유용성 점수 컬럼이 데이터에 없습니다.")


In [ ]:
## 5. 사용자별 리뷰 품질 집계

def calculate_user_review_quality(interaction_df, user_processed):
    """
    사용자별 리뷰 품질 지표 계산
    
    Args:
        interaction_df (pd.DataFrame): 상호작용 데이터프레임
        user_processed (pd.DataFrame): 사용자 데이터프레임
        
    Returns:
        pd.DataFrame: 리뷰 품질 지표가 포함된 사용자 데이터프레임
    """
    user_df = user_processed.copy()
    
    # 필요한 컬럼 확인
    required_cols = ['user_idx', 'useful', 'funny', 'cool', 'text_length']
    missing_cols = [col for col in required_cols if col not in interaction_df.columns]
    
    if missing_cols:
        print(f"누락된 컬럼이 있습니다: {missing_cols}")
        print("기본 신뢰도 점수를 사용합니다.")
        return user_df
    
    # 사용자별 리뷰 품질 지표 집계
    user_review_quality = interaction_df.groupby('user_idx').agg(
        avg_useful=('useful', 'mean'),
        avg_funny=('funny', 'mean'),
        avg_cool=('cool', 'mean'),
        avg_text_length=('text_length', 'mean'),
        max_useful=('useful', 'max'),
        max_funny=('funny', 'max'),
        max_cool=('cool', 'max'),
        total_useful=('useful', 'sum'),
        total_funny=('funny', 'sum'),
        total_cool=('cool', 'sum'),
        review_count=('user_idx', 'count')
    ).reset_index()
    
    # 사용자 데이터프레임과 병합
    user_df = user_df.merge(user_review_quality, on='user_idx', how='left')
    
    # 결측치 채우기
    cols_to_fill = user_review_quality.columns.drop('user_idx')
    user_df[cols_to_fill] = user_df[cols_to_fill].fillna(0)
    
    return user_df

# 사용자별 리뷰 품질 집계
user_review_quality_df = calculate_user_review_quality(interaction_df, user_processed)
print(f"리뷰 품질 지표가 추가된 사용자 데이터프레임 크기: {user_review_quality_df.shape}")
print(user_review_quality_df.head())

In [ ]:
## 6. 신뢰도 점수 계산 함수 구현

def calculate_trust_score_simple(user_df):
    """
    간단한 사용자 신뢰도 점수 계산 (기본 버전)
    
    Args:
        user_df (pd.DataFrame): 사용자 데이터프레임
        
    Returns:
        pd.DataFrame: 신뢰도 점수가 추가된 사용자 데이터프레임
    """
    df = user_df.copy()
    
    # 간단한 신뢰도 점수: 팬 수 + 리뷰 수 + 총 컴플리먼트
    if 'fans' in df.columns and 'review_count' in df.columns and 'total_compliments' in df.columns:
        df['trust_score_simple'] = df['fans'] * 2 + df['review_count'] + df['total_compliments'] * 0.5
    else:
        print("필요한 컬럼(fans, review_count, total_compliments)이 없습니다.")
        df['trust_score_simple'] = 0
    
    return df

def calculate_trust_score_advanced(user_df):
    """
    고급 사용자 신뢰도 점수 계산 (리뷰 품질 지표 포함)
    
    Args:
        user_df (pd.DataFrame): 사용자 데이터프레임
        
    Returns:
        pd.DataFrame: 신뢰도 점수가 추가된 사용자 데이터프레임
    """
    df = user_df.copy()
    
    # 필요한 컬럼 확인
    required_cols = [
        'fans', 'review_count', 'total_compliments', 
        'total_useful', 'total_funny', 'total_cool',
        'avg_text_length', 'activity_intensity'
    ]
    missing_cols = [col for col in required_cols if col not in df.columns]
    
    if missing_cols:
        print(f"누락된 컬럼이 있습니다: {missing_cols}")
        print("간단한 신뢰도 점수를 대신 사용합니다.")
        return calculate_trust_score_simple(df)
    
    # 고급 신뢰도 점수 계산 (가중치 사용)
    df['trust_score_advanced'] = (
        df['fans'] * 2 +                   # 팬 수 (가중치 2)
        df['review_count'] * 1 +           # 리뷰 수 (가중치 1)
        df['total_compliments'] * 0.5 +    # 총 컴플리먼트 (가중치 0.5)
        df['total_useful'] * 3 +           # 총 유용성 점수 (가중치 3)
        df['total_funny'] * 1 +            # 총 재미 점수 (가중치 1)
        df['total_cool'] * 2 +             # 총 쿨 점수 (가중치 2)
        df['avg_text_length'] * 0.1 +      # 평균 리뷰 길이 (가중치 0.1)
        df['activity_intensity'] * 100     # 활동 강도 (가중치 100)
    )
    
    return df

def normalize_trust_scores(user_df, score_cols):
    """
    신뢰도 점수 정규화 (0-1 범위)
    
    Args:
        user_df (pd.DataFrame): 사용자 데이터프레임
        score_cols (list): 정규화할 점수 컬럼 리스트
        
    Returns:
        pd.DataFrame: 정규화된 점수가 추가된 사용자 데이터프레임
    """
    df = user_df.copy()
    
    for col in score_cols:
        if col in df.columns:
            # 이상치의 영향을 줄이기 위해 99 퍼센타일로 상한 설정
            upper_bound = df[col].quantile(0.99)
            capped_scores = df[col].clip(upper=upper_bound)
            
            # Min-Max 정규화
            scaler = MinMaxScaler()
            df[f'{col}_normalized'] = scaler.fit_transform(capped_scores.values.reshape(-1, 1))
    
    return df

# 신뢰도 점수 계산
user_trust_df = calculate_trust_score_simple(user_review_quality_df)
user_trust_df = calculate_trust_score_advanced(user_trust_df)
user_trust_df = normalize_trust_scores(user_trust_df, ['trust_score_simple', 'trust_score_advanced'])

print("\n=== 신뢰도 점수 통계 ===")
print("\n간단한 신뢰도 점수:")
print(user_trust_df['trust_score_simple'].describe())
print("\n고급 신뢰도 점수:")
print(user_trust_df['trust_score_advanced'].describe())
print("\n정규화된 간단한 신뢰도 점수:")
print(user_trust_df['trust_score_simple_normalized'].describe())
print("\n정규화된 고급 신뢰도 점수:")
print(user_trust_df['trust_score_advanced_normalized'].describe())

# 신뢰도 점수 분포 시각화
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 간단한 신뢰도 점수 분포
sns.histplot(user_trust_df['trust_score_simple'], bins=50, kde=True, ax=axes[0, 0])
axes[0, 0].set_title('간단한 신뢰도 점수 분포')
axes[0, 0].set_xlabel('신뢰도 점수')
axes[0, 0].set_ylabel('사용자 수')
axes[0, 0].grid(True, alpha=0.3)

# 고급 신뢰도 점수 분포
sns.histplot(user_trust_df['trust_score_advanced'], bins=50, kde=True, ax=axes[0, 1])
axes[0, 1].set_title('고급 신뢰도 점수 분포')
axes[0, 1].set_xlabel('신뢰도 점수')
axes[0, 1].set_ylabel('사용자 수')
axes[0, 1].grid(True, alpha=0.3)

# 정규화된 간단한 신뢰도 점수 분포
sns.histplot(user_trust_df['trust_score_simple_normalized'], bins=50, kde=True, ax=axes[1, 0])
axes[1, 0].set_title('정규화된 간단한 신뢰도 점수 분포')
axes[1, 0].set_xlabel('신뢰도 점수 (0-1)')
axes[1, 0].set_ylabel('사용자 수')
axes[1, 0].grid(True, alpha=0.3)

# 정규화된 고급 신뢰도 점수 분포
sns.histplot(user_trust_df['trust_score_advanced_normalized'], bins=50, kde=True, ax=axes[1, 1])
axes[1, 1].set_title('정규화된 고급 신뢰도 점수 분포')
axes[1, 1].set_xlabel('신뢰도 점수 (0-1)')
axes[1, 1].set_ylabel('사용자 수')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
## 7. 신뢰도 점수와 다른 지표 간의 상관관계 분석

# 상관관계 분석을 위한 컬럼 선택
correlation_cols = [
    'review_count', 'fans', 'total_compliments', 
    'activity_intensity', 'avg_useful', 'avg_text_length',
    'trust_score_simple', 'trust_score_advanced'
]

# 상관관계 행렬 계산
correlation_matrix = user_trust_df[correlation_cols].corr()

# 히트맵 시각화
plt.figure(figsize=(12, 10))
sns.heatmap(
    correlation_matrix, 
    annot=True, 
    fmt='.2f', 
    cmap='coolwarm', 
    vmin=-1, 
    vmax=1,
    linewidths=0.5
)
plt.title('사용자 특성 및 신뢰도 점수 간의 상관관계')
plt.tight_layout()
plt.show()

## 8. 신뢰도 점수별 사용자 그룹화 및 분석

def categorize_users_by_trust(user_df, score_col, n_groups=4):
    """
    신뢰도 점수를 기반으로 사용자 그룹화
    
    Args:
        user_df (pd.DataFrame): 사용자 데이터프레임
        score_col (str): 사용할 점수 컬럼
        n_groups (int): 그룹 수
        
    Returns:
        pd.DataFrame: 그룹이 추가된 사용자 데이터프레임
    """
    df = user_df.copy()
    
    # 그룹 경계 계산
    quantiles = np.linspace(0, 1, n_groups + 1)
    boundaries = [df[score_col].quantile(q) for q in quantiles]
    
    # 그룹 레이블 생성
    labels = [f'G{i+1}' for i in range(n_groups)]
    
    # 사용자 그룹화
    df['trust_group'] = pd.cut(
        df[score_col], 
        bins=boundaries, 
        labels=labels, 
        include_lowest=True
    )
    
    return df

# 정규화된 고급 신뢰도 점수로 사용자 그룹화
user_trust_df = categorize_users_by_trust(
    user_trust_df, 
    'trust_score_advanced_normalized', 
    n_groups=4
)

# 그룹별 사용자 수
trust_group_counts = user_trust_df['trust_group'].value_counts().sort_index()
print("\n=== 신뢰도 그룹별 사용자 수 ===")
print(trust_group_counts)

# 그룹별 특성 평균
trust_group_means = user_trust_df.groupby('trust_group').agg({
    'review_count': 'mean',
    'fans': 'mean',
    'total_compliments': 'mean',
    'activity_intensity': 'mean',
    'avg_useful': 'mean',
    'avg_text_length': 'mean',
    'trust_score_advanced': 'mean'
}).round(2)

print("\n=== 신뢰도 그룹별 특성 평균 ===")
print(trust_group_means)

# 그룹별 특성 시각화
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

metrics = [
    'review_count', 'fans', 'total_compliments', 
    'activity_intensity', 'avg_useful', 'avg_text_length'
]

for i, metric in enumerate(metrics):
    sns.barplot(x='trust_group', y=metric, data=user_trust_df, ax=axes[i])
    axes[i].set_title(f'신뢰도 그룹별 평균 {metric}')
    axes[i].set_xlabel('신뢰도 그룹')
    axes[i].set_ylabel(f'평균 {metric}')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
## 9. 최종 신뢰도 점수 선택 및 저장

# 최종 신뢰도 점수 선택 (정규화된 고급 신뢰도 점수)
user_trust_df['final_trust_score'] = user_trust_df['trust_score_advanced_normalized']

# 필요한 컬럼만 선택하여 최종 데이터프레임 생성
final_trust_df = user_trust_df[['user_idx', 'final_trust_score', 'trust_group']].copy()

print("\n=== 최종 신뢰도 점수 통계 ===")
print(final_trust_df['final_trust_score'].describe())
print(f"\n총 사용자 수: {len(final_trust_df):,}")

# 최종 신뢰도 점수 저장
with open(os.path.join(TRUST_DIR, 'user_trust_scores.pkl'), 'wb') as f:
    pickle.dump(final_trust_df, f)
print(f"\n최종 신뢰도 점수 저장 완료: {os.path.join(TRUST_DIR, 'user_trust_scores.pkl')}")

# 전체 신뢰도 데이터프레임 저장 (참조용)
with open(os.path.join(TRUST_DIR, 'user_trust_full.pkl'), 'wb') as f:
    pickle.dump(user_trust_df, f)
print(f"전체 신뢰도 데이터 저장 완료: {os.path.join(TRUST_DIR, 'user_trust_full.pkl')}")


In [ ]:
## 10. 신뢰도 점수를 활용한 가중치 함수 구현

def trust_weight_linear(trust_score):
    """
    선형 신뢰도 가중치 함수
    
    Args:
        trust_score (float): 신뢰도 점수 (0-1)
        
    Returns:
        float: 가중치 (0.5-1.5)
    """
    # 신뢰도 점수에 따라 0.5에서 1.5 사이의 가중치 반환
    return 0.5 + trust_score * 1.0

def trust_weight_sigmoid(trust_score, k=10):
    """
    시그모이드 신뢰도 가중치 함수
    
    Args:
        trust_score (float): 신뢰도 점수 (0-1)
        k (float): 시그모이드 기울기
        
    Returns:
        float: 가중치 (0.5-1.5)
    """
    # 시그모이드 함수를 사용하여 0.5에서 1.5 사이의 가중치 반환
    x = (trust_score - 0.5) * k
    sigmoid = 1 / (1 + np.exp(-x))
    return 0.5 + sigmoid * 1.0

def trust_weight_step(trust_score, steps=4):
    """
    계단식 신뢰도 가중치 함수
    
    Args:
        trust_score (float): 신뢰도 점수 (0-1)
        steps (int): 계단 수
        
    Returns:
        float: 가중치 (0.5-1.5)
    """
    # 계단식 함수를 사용하여 0.5에서 1.5 사이의 가중치 반환
    step_size = 1.0 / steps
    step = int(trust_score / step_size)
    step = min(step, steps - 1)  # 최대 계단 수 제한
    return 0.5 + step * (1.0 / (steps - 1))

# 가중치 함수 시각화
trust_scores = np.linspace(0, 1, 100)
linear_weights = [trust_weight_linear(score) for score in trust_scores]
sigmoid_weights = [trust_weight_sigmoid(score) for score in trust_scores]
step_weights = [trust_weight_step(score) for score in trust_scores]

plt.figure(figsize=(12, 6))
plt.plot(trust_scores, linear_weights, label='선형 가중치')
plt.plot(trust_scores, sigmoid_weights, label='시그모이드 가중치')
plt.plot(trust_scores, step_weights, label='계단식 가중치')
plt.title('신뢰도 점수에 따른 가중치 함수')
plt.xlabel('신뢰도 점수')
plt.ylabel('가중치')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# 실제 사용자 데이터로 가중치 분포 확인
user_trust_df['linear_weight'] = user_trust_df['final_trust_score'].apply(trust_weight_linear)
user_trust_df['sigmoid_weight'] = user_trust_df['final_trust_score'].apply(trust_weight_sigmoid)
user_trust_df['step_weight'] = user_trust_df['final_trust_score'].apply(trust_weight_step)

# 가중치 분포 시각화
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

sns.histplot(user_trust_df['linear_weight'], bins=50, kde=True, ax=axes[0])
axes[0].set_title('선형 가중치 분포')
axes[0].set_xlabel('가중치')
axes[0].set_ylabel('사용자 수')
axes[0].grid(True, alpha=0.3)

sns.histplot(user_trust_df['sigmoid_weight'], bins=50, kde=True, ax=axes[1])
axes[1].set_title('시그모이드 가중치 분포')
axes[1].set_xlabel('가중치')
axes[1].set_ylabel('사용자 수')
axes[1].grid(True, alpha=0.3)

sns.histplot(user_trust_df['step_weight'], bins=50, kde=True, ax=axes[2])
axes[2].set_title('계단식 가중치 분포')
axes[2].set_xlabel('가중치')
axes[2].set_ylabel('사용자 수')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
## 11. 가중치 함수 저장

# 가중치 함수 저장
weight_functions = {
    'linear': trust_weight_linear,
    'sigmoid': trust_weight_sigmoid,
    'step': trust_weight_step
}

with open(os.path.join(TRUST_DIR, 'trust_weight_functions.pkl'), 'wb') as f:
    pickle.dump(weight_functions, f)
print(f"가중치 함수 저장 완료: {os.path.join(TRUST_DIR, 'trust_weight_functions.pkl')}")

# 최종 결과 확인
print("\n=== 신뢰도 점수 및 가중치 요약 ===")
print(f"총 사용자 수: {len(user_trust_df):,}")
print(f"신뢰도 그룹 수: {len(trust_group_counts):,}")
print(f"최종 신뢰도 점수 범위: {user_trust_df['final_trust_score'].min():.4f} - {user_trust_df['final_trust_score'].max():.4f}")
print(f"평균 신뢰도 점수: {user_trust_df['final_trust_score'].mean():.4f}")
print(f"가중치 범위 (선형): {user_trust_df['linear_weight'].min():.4f} - {user_trust_df['linear_weight'].max():.4f}")
print(f"가중치 범위 (시그모이드): {user_trust_df['sigmoid_weight'].min():.4f} - {user_trust_df['sigmoid_weight'].max():.4f}")
print(f"가중치 범위 (계단식): {user_trust_df['step_weight'].min():.4f} - {user_trust_df['step_weight'].max():.4f}")

print("\n사용자 신뢰도 점수 계산 및 분석 완료!")